# GPU

### 1 查看GPU状态

In [1]:
!nvidia-smi

Thu May 30 22:25:20 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:18:00.0 Off |                    0 |
| N/A   77C    P0            221W /  250W |   21798MiB /  40960MiB |     88%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import torch
from torch import nn

选择使用cpu或gpu

In [3]:
torch.device('cpu')
torch.cuda.device('cuda')
torch.cuda.device('cuda:1')

device(type='cpu')

In [6]:
torch.cuda.device_count()

4

允许在请求GPU不存在的情况下运行代码

In [7]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [8]:
def try_all_gpus():
    devices = [
        torch.device(f'cuda:{i}') for i in range
        (torch.cuda.device_count())
    ]
    return devices if devices else [torch.device('cpu')]

In [9]:
try_gpu()

device(type='cuda', index=0)

In [10]:
try_gpu(10)

device(type='cpu')

In [11]:
try_all_gpus()

[device(type='cuda', index=0),
 device(type='cuda', index=1),
 device(type='cuda', index=2),
 device(type='cuda', index=3)]

### 2 对应设备创建张量

默认在cpu上

In [13]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

指定在gpu上创建

In [14]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [15]:
Y = torch.ones(2, 3, device=try_gpu(3))
Y

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:3')

只有 X和Y 在同一块GPU上才能进行运算，将X复制移动到3号gpu上于Y运算

In [18]:
Z = X.cuda(3)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:3')


In [19]:
Y+Z

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:3')

### 3.神经网络中使用GPU

In [20]:
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.2573],
        [-0.2573]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [22]:
net[0].weight.data.device

device(type='cuda', index=0)